In [37]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import re
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
import pyLDAvis
import pyLDAvis.sklearn
from gensim.models.coherencemodel import CoherenceModel
from gensim.corpora import Dictionary
from gensim.models.ldamodel import LdaModel
from gensim.matutils import Sparse2Corpus

pyLDAvis.enable_notebook()
nltk.download('wordnet')
nltk.download('stopwords')
#sklearn.decomposition.LatentDirichletAllocation

[nltk_data] Downloading package wordnet to /home/bruno/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /home/bruno/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [38]:
data = pd.read_csv('data.csv')
data.fillna('', inplace=True)
data.head()

,contest,problem_name,problem_statement,problem_tags
0,325,A,You are given n rectangles. The corners of rec...,"implementation,*1500"
1,325,B,Daniel is organizing a football tournament. He...,"binarysearch,math,*1800"
2,325,C,Piegirl has found a monster and a book about m...,"dfsandsimilar,graphs,shortestpaths,*2600"
3,325,D,"In a far away land, there exists a planet shap...","dsu,*2900"
4,325,E,Piegirl found the red button. You have one las...,"combinatorics,dfsandsimilar,dsu,graphs,greedy,..."


In [39]:
print(f'quantidade de problemas: {data.shape[0]}')
print(f'quantidade de contests: {len(data.contest.unique())}')
print(f'problemas repetidos: {data.shape[0] - len(data.problem_statement.unique())}')

data.drop_duplicates(subset='problem_statement', inplace=True)

quantidade de problemas: 8343
quantidade de contests: 1424
problemas repetidos: 1524


## Pre-processamento

In [78]:
sw = stopwords.words('english')
sw.append(['input', 'output', 'th'])
lemma = WordNetLemmatizer()
text = data.problem_statement

# Problem_statement
print(text[8341])
print('------------')
text = text.apply(lambda x: re.sub('(\$+)(?:(?!\1)[\s\S])*\1', ' ',x))
print(text[8341])
text = text.apply(lambda x: re.sub('[,|.|(|)|$|!|?|!]',' ',x)) # removendo caracteres especiais
text = text.apply(lambda x: [i for i in x.split() if i.lower() not in sw]) # removendo stop-words
text = text.apply(lambda x: ' '.join([lemma.lemmatize(i) for i in x])) # lemmatizando todo

# Topics
topics = data.problem_tags
topics = topics.apply(lambda x: re.sub('[*][0-9]+','',x)) # remove os ratings *800 etc
topics = topics.apply(lambda x: [i for i in x.split(',') if i != '']) # tira strings vazias

<>:9: DeprecationWarning: invalid escape sequence '\$'
<>:9: DeprecationWarning: invalid escape sequence '\$'
/tmp/ipykernel_34414/3887595552.py:9: DeprecationWarning: invalid escape sequence '\$'
  text = text.apply(lambda x: re.sub('(\$+)(?:(?!\1)[\s\S])*\1', ' ',x))


At first, let's define function $$$f(x)$$$ as follows: $$$$$$ \begin{matrix} f(x) & = & \left\{ \begin{matrix} \frac{x}{2} & \mbox{if } x \text{ is even} \\ x - 1 & \mbox{otherwise } \end{matrix} \right. \end{matrix} $$$$$$

We can see that if we choose some value $$$v$$$ and will apply function $$$f$$$ to it, then apply $$$f$$$ to $$$f(v)$$$, and so on, we'll eventually get $$$1$$$. Let's write down all values we get in this process in a list and denote this list as $$$path(v)$$$. For example, $$$path(1) = [1]$$$, $$$path(15) = [15, 14, 7, 6, 3, 2, 1]$$$, $$$path(32) = [32, 16, 8, 4, 2, 1]$$$.

Let's write all lists $$$path(x)$$$ for every $$$x$$$ from $$$1$$$ to $$$n$$$. The question is next: what is the maximum value $$$y$$$ such that $$$y$$$ is contained in at least $$$k$$$ different lists $$$path(x)$$$?

Formally speaking, you need to find maximum $$$y$$$ such that $$$\left| \{ x ~|~ 1 \le x \le n, y \in path(x) \} \right| \ge k$$$.

The first line contains two integers $$$n$$$ an

In [41]:
text

0       given n rectangle corner rectangle integer coo...
1       Daniel organizing football tournament come fol...
2       Piegirl found monster book monster pie reading...
3       far away land exists planet shaped like cylind...
4       Piegirl found red button one last chance chang...
                              ...                        
8338    n block arranged row numbered left right start...
8339    map capital Berland viewed infinite coordinate...
8340    play strategic video game yeah ran good proble...
8341    first let's define function f x follows: \begi...
8342    Recently lot student enrolled Berland State Un...
Name: problem_statement, Length: 6819, dtype: object

In [42]:
topics

0                                        [implementation]
1                                    [binarysearch, math]
2                  [dfsandsimilar, graphs, shortestpaths]
3                                                   [dsu]
4       [combinatorics, dfsandsimilar, dsu, graphs, gr...
                              ...                        
8338                                       [greedy, math]
8339       [bruteforce, geometry, greedy, implementation]
8340    [datastructures, dp, greedy, implementation, s...
8341              [binarysearch, combinatorics, dp, math]
8342                                         [bruteforce]
Name: problem_tags, Length: 6819, dtype: object

In [43]:
lista = topics.to_list()

d = dict()
for x in lista:
    for y in x:
        if y not in d:
            d[y]= 1
        else:
            d[y]+=1

x= sorted(d.items(), key=lambda item:item[1], reverse=True)

for i in x:
    print(i)



('implementation', 1970)
('math', 1733)
('greedy', 1645)
('dp', 1373)
('datastructures', 1076)
('bruteforce', 1020)
('constructivealgorithms', 981)
('graphs', 720)
('sortings', 658)
('binarysearch', 652)
('dfsandsimilar', 590)
('trees', 511)
('strings', 484)
('numbertheory', 453)
('combinatorics', 390)
('*specialproblem', 367)
('twopointers', 310)
('bitmasks', 300)
('geometry', 291)
('dsu', 210)
('shortestpaths', 174)
('divideandconquer', 167)
('probabilities', 162)
('hashing', 136)
('games', 131)
('interactive', 114)
('flows', 99)
('matrices', 87)
('stringsuffixstructures', 64)
('fft', 56)
('graphmatchings', 54)
('ternarysearch', 40)
('meet-in-the-middle', 33)
('expressionparsing', 32)
('2-sat', 19)
('chineseremaindertheorem', 10)
('schedules', 5)


## Vetorizando

In [44]:
max_df = 0.8
min_df = 0.05
vec = CountVectorizer(max_df=max_df, min_df=min_df, ngram_range=(1,3))

In [45]:
def vect2gensim(vectorizer, dtmatrix):
    # transform sparse matrix into gensim corpus and dictionary
    corpus_vect_gensim = Sparse2Corpus(dtmatrix, documents_columns=False)
    dictionary = Dictionary.from_corpus(corpus_vect_gensim,
            id2word=dict((id, word) for word, id in vectorizer.vocabulary_.items()))
    return (corpus_vect_gensim, dictionary)



In [46]:
X = vec.fit_transform(text)
X.shape
X = vect2gensim(vec, X)

## Agrupando

In [47]:
# Pegando quantidade de topicos

total_topics = set()
topics = topics.apply(lambda x: [total_topics.add(i) for i in x])

In [48]:
n_total_topics = len(total_topics)
print(f'Número de Tópicos: {n_total_topics}')
total_topics

Número de Tópicos: 37


{'*specialproblem',
 '2-sat',
 'binarysearch',
 'bitmasks',
 'bruteforce',
 'chineseremaindertheorem',
 'combinatorics',
 'constructivealgorithms',
 'datastructures',
 'dfsandsimilar',
 'divideandconquer',
 'dp',
 'dsu',
 'expressionparsing',
 'fft',
 'flows',
 'games',
 'geometry',
 'graphmatchings',
 'graphs',
 'greedy',
 'hashing',
 'implementation',
 'interactive',
 'math',
 'matrices',
 'meet-in-the-middle',
 'numbertheory',
 'probabilities',
 'schedules',
 'shortestpaths',
 'sortings',
 'strings',
 'stringsuffixstructures',
 'ternarysearch',
 'trees',
 'twopointers'}

In [54]:
n_components = n_total_topics
max_iter = 100

id2word = X[1]
corpus = X[0]

lda = LdaModel(corpus=corpus, num_topics=n_components, id2word=id2word, iterations=max_iter)

In [50]:
# lda_matrix = lda.fit_transform(X)



In [51]:
# top_word = lda.components_
# terms = vec.get_feature_names_out()

# for i, comp in enumerate(top_word):
#     top_terms_key = zip(terms, comp)
#     top_terms_key= sorted(top_terms_key, key = lambda t: t[1], reverse=True)[:10]
#     top_terms_list= list(dict(top_terms_key).keys())
#     print("Topic "+str(i)+": ",top_terms_list)

## Validando

In [52]:
new_model = CoherenceModel(model=lda, texts=text, corpus=corpus, dictionary=id2word, coherence="u_mass")
print(new_model.get_coherence())

# todo -> entender metrica de coerencia
#      -> usar dentro dos atributos o que ja foi calculado no CountVectorizer

-1.188056306630878


## Visualizando

In [55]:
import pyLDAvis
import pyLDAvis.gensim
pyLDAvis.enable_notebook()

panel = pyLDAvis.gensim.prepare(lda, corpus, dictionary=id2word)
panel
# todo 
# mudar para o modelo do gensim

/home/bruno/Desktop/PIBIC/venv/lib/python3.10/site-packages/pyLDAvis/_prepare.py:244: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  by='saliency', ascending=False).head(R).drop('saliency', 1)


PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
17    -0.216424 -0.059456       1        1  7.545809
0      0.094207  0.008006       2        1  6.991268
23     0.010180 -0.004915       3        1  5.315583
1      0.106739  0.006809       4        1  4.293599
28    -0.184024  0.128371       5        1  4.082773
29     0.097929  0.053383       6        1  3.988818
14     0.057345  0.067769       7        1  3.716175
11     0.054097  0.030149       8        1  3.665124
32    -0.037260 -0.043119       9        1  3.552772
22    -0.085743  0.060921      10        1  3.515665
13     0.111374 -0.035944      11        1  3.406708
9      0.070186  0.018967      12        1  3.222534
24     0.130018 -0.005656      13        1  3.087354
34    -0.172375  0.137325      14        1  3.049027
26     0.083391 -0.008987      15        1  2.997623
12    -0.084006  0.120003      16        1  2.640441
7      0.038374 -0.001079      17        1  2.556178
16     0.110232 -0.051646      18        1  2.532445
35     0.029618 -0.013891      19        1  2.366820
31     0.087953  0.002922      20        1  2.351869
33    -0.026663  0.062076      21        1  2.270833
25     0.058205  0.057637      22        1  2.219259
10    -0.088766  0.051847      23        1  2.017108
21     0.049932 -0.015997      24        1  1.996054
30     0.109192 -0.026936      25        1  1.987790
5      0.027167  0.015460      26        1  1.842659
18     0.005816 -0.036010      27        1  1.806013
36     0.080761  0.007404      28        1  1.588713
6      0.031198 -0.021786      29        1  1.525113
3     -0.130108 -0.206279      30        1  1.481551
4     -0.010769  0.041642      31        1  1.349140
27    -0.036569 -0.109795      32        1  1.293654
15    -0.039929  0.020266      33        1  0.990305
8     -0.149750  0.000557      34        1  0.934642
19    -0.027551 -0.008025      35        1  0.756434
20    -0.030823 -0.076114      36        1  0.533993
2     -0.123155 -0.165877      37        1  0.528157, topic_info=          Term          Freq         Total Category  logprob  loglift
235     string   7518.000000   7518.000000  Default  30.0000  30.0000
224         le  14082.000000  14082.000000  Default  29.0000  29.0000
101       cell   3825.000000   3825.000000  Default  28.0000  28.0000
144       test   6066.000000   6066.000000  Default  27.0000  27.0000
315  test case   4537.000000   4537.000000  Default  26.0000  26.0000
..         ...           ...           ...      ...      ...      ...
139     second     33.114550   9831.298862  Topic37  -4.5921  -0.4498
20         two     31.204097   9741.819111  Topic37  -4.6515  -0.5001
224         le     31.599955  14082.591427  Topic37  -4.6389  -0.8560
0        given     28.188256   5457.533846  Topic37  -4.7532  -0.0223
235     string     26.587230   7518.098344  Topic37  -4.8117  -0.4011

[1825 rows x 6 columns], token_table=      Topic      Freq  Term
term                       
382       1  0.014302   000
382       2  0.025171   000
382       3  0.029176   000
382       4  0.034324   000
382       5  0.025743   000
...     ...       ...   ...
65       33  0.004960  zero
65       34  0.005669  zero
65       35  0.014172  zero
65       36  0.003543  zero
65       37  0.002834  zero

[13105 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[18, 1, 24, 2, 29, 30, 15, 12, 33, 23, 14, 10, 25, 35, 27, 13, 8, 17, 36, 32, 34, 26, 11, 22, 31, 6, 19, 37, 7, 4, 5, 28, 16, 9, 20, 21, 3])